In [ ]:
import tensorflow as tf
import numpy as np
import os
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional, Embedding
from string import punctuation

In [ ]:
sequence_length = 100
BATCH_SIZE = 128
EPOCHS = 30
# dataset file path
FILE_PATH = "/content/Full_Dataset.txt"
BASENAME = os.path.basename(FILE_PATH)
text = open(FILE_PATH, encoding="utf-8").read()
text = open(FILE_PATH, encoding="utf-8").read()
text = text.lower()
text = text.translate(str.maketrans("", "", punctuation))

In [ ]:
import re
 
text = re.sub(r'[.,0145«»"\'-?:!;–—…﻿tacehopty]', '', text)

In [ ]:
# print some stats
n_chars = len(text)
vocab = ''.join(sorted(set(text)))
print("unique_chars:", vocab)
n_unique_chars = len(vocab)
print("Number of characters:", n_chars)
print("Number of unique characters:", n_unique_chars)

unique_chars: 
 əабвгдежзийклмнопрстуфхцчшщъыьэюяіғқңүұһәө
Number of characters: 495346
Number of unique characters: 44


In [ ]:
# dictionary that converts characters to integers
char2int = {c: i for i, c in enumerate(vocab)}
# dictionary that converts integers to characters
int2char = {i: c for i, c in enumerate(vocab)}

In [ ]:
# save these dictionaries for later generation
pickle.dump(char2int, open(f"{BASENAME}-char2int.pickle", "wb"))
pickle.dump(int2char, open(f"{BASENAME}-int2char.pickle", "wb"))

In [ ]:
# convert all text into integers
encoded_text = np.array([char2int[c] for c in text])
# construct tf.data.Dataset object
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)
# print first 5 characters
for char in char_dataset.take(8):
    print(char.numpy(), int2char[char.numpy()])

4 б
3 а
12 й
1  
4 б
17 о
14 л
36 ғ


In [ ]:
sequences = char_dataset.batch(2*sequence_length + 1, drop_remainder=True)

for sequence in sequences.take(2):
    print(''.join([int2char[i] for i in sequence.numpy()]))

бай болған дəулеті мол тілеуберді
байлықпенен меңгерген тамам елді
төрт түлік мал өргенде құрттай қайнап
құмырсқадай қыбырлап жапқан жерді
торқасқалы қазмойын жылқыларын
екіүш мыңдап он шақты қосқа бөл
ді
түйелері түздерде бота тауып
қойы жүрген жерінен қозы терді
жас кезінде қыз таңдап кезіп елді
өз бойына лайықты іздеп теңді
мың жылқыны біржола матап беріп
алтынай дейтін сұлуды алып еді
тоқсан нарғ


In [ ]:
def split_sample(sample):
    ds = tf.data.Dataset.from_tensors((sample[:sequence_length], sample[sequence_length]))
    for i in range(1, (len(sample)-1) // 2):
        input_ = sample[i: i+sequence_length]
        target = sample[i+sequence_length]
        # extend the dataset with these samples by concatenate() method
        other_ds = tf.data.Dataset.from_tensors((input_, target))
        ds = ds.concatenate(other_ds)
    return ds

dataset = sequences.flat_map(split_sample)

In [ ]:
def one_hot_samples(input_, target):
    # onehot encode the inputs and the targets
    return tf.one_hot(input_, n_unique_chars), tf.one_hot(target, n_unique_chars)

dataset = dataset.map(one_hot_samples)

In [ ]:
# print first 2 samples
for element in dataset.take(2):
    print("Input:", ''.join([int2char[np.argmax(char_vector)] for char_vector in element[0].numpy()]))
    print("Target:", int2char[np.argmax(element[1].numpy())])
    print("Input shape:", element[0].shape)
    print("Target shape:", element[1].shape)
    print("="*50, "\n")

Input: бай болған дəулеті мол тілеуберді
байлықпенен меңгерген тамам елді
төрт түлік мал өргенде құрттай қа
Target: й
Input shape: (100, 44)
Target shape: (44,)

Input: ай болған дəулеті мол тілеуберді
байлықпенен меңгерген тамам елді
төрт түлік мал өргенде құрттай қай
Target: н
Input shape: (100, 44)
Target shape: (44,)



In [ ]:
ds = dataset.repeat().shuffle(1024).batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
model = Sequential([
    LSTM(256, input_shape=(sequence_length, n_unique_chars), return_sequences=True),
    Dropout(0.25),
    LSTM(256),
    Dense(n_unique_chars, activation="softmax"),
])

In [ ]:
model_weights_path = f"results/{BASENAME}-{sequence_length}.h5"
model.summary()
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 100, 256)          308224    
                                                                 
 dropout_1 (Dropout)         (None, 100, 256)          0         
                                                                 
 lstm_3 (LSTM)               (None, 256)               525312    
                                                                 
 dense_1 (Dense)             (None, 44)                11308     
                                                                 
Total params: 844,844
Trainable params: 844,844
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# make results folder if does not exist yet
if not os.path.isdir("results"):
    os.mkdir("results")
model.fit(ds, steps_per_epoch=(len(encoded_text) - sequence_length) // BATCH_SIZE, epochs=EPOCHS)
model.save(model_weights_path)

Epoch 1/30
3869/3869 [==============================] - 128s 32ms/step - loss: 2.0444 - accuracy: 0.3513
Epoch 2/30
3869/3869 [==============================] - 124s 32ms/step - loss: 1.7490 - accuracy: 0.4344
Epoch 3/30
3869/3869 [==============================] - 125s 32ms/step - loss: 1.6247 - accuracy: 0.4736
Epoch 4/30
3869/3869 [==============================] - 124s 32ms/step - loss: 1.5332 - accuracy: 0.5026
Epoch 5/30
3869/3869 [==============================] - 124s 32ms/step - loss: 1.4550 - accuracy: 0.5265
Epoch 6/30
3869/3869 [==============================] - 125s 32ms/step - loss: 1.3884 - accuracy: 0.5473
Epoch 7/30
3869/3869 [==============================] - 124s 32ms/step - loss: 1.3297 - accuracy: 0.5665
Epoch 8/30
3869/3869 [==============================] - 124s 32ms/step - loss: 1.2761 - accuracy: 0.5841
Epoch 9/30
3869/3869 [==============================] - 125s 32ms/step - loss: 1.2339 - accuracy: 0.5960
Epoch 10/30
3869/3869 [==============================] 

**Generating Text**

In [41]:
import numpy as np
import pickle
import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Activation
import os

sequence_length = 100
# dataset file path
FILE_PATH = "/content/Full_Dataset.txt"
BASENAME = os.path.basename(FILE_PATH)

seed = "ұл туды деп шашулар шашылған жоқ\n"

# load vocab dictionaries
char2int = pickle.load(open(f"{BASENAME}-char2int.pickle", "rb"))
int2char = pickle.load(open(f"{BASENAME}-int2char.pickle", "rb"))
vocab_size = len(char2int)

model = Sequential([
    LSTM(256, input_shape=(sequence_length, vocab_size), return_sequences=True),
    Dropout(0.25),
    LSTM(256),
    Dense(vocab_size, activation="softmax"),
])

model.load_weights(f"results/{BASENAME}-{sequence_length}.h5")

s = seed
n_chars = 400
generated = ""
for i in tqdm.tqdm(range(n_chars), "Generating text"):
    # make the input sequence
    X = np.zeros((1, sequence_length, vocab_size))
    for t, char in enumerate(seed):
      X[0, (sequence_length - len(seed)) + t, char2int[char]] = 1
    # predict the next character
    predicted = model.predict(X, verbose=0)[0]
    # converting the vector to an integer
    next_index = np.argmax(predicted)
    # converting the integer to a character
    next_char = int2char[next_index]
    # add the character to results
    generated += next_char
    # shift seed and the predicted character
    seed = seed[1:] + next_char

print("Seed:", s)
print("Generated text:")
print(generated)

Generating text: 100%|██████████| 400/400 [00:22<00:00, 17.55it/s]

Seed: ұл туды деп шашулар шашылған жоқ

Generated text:
бұл іске түсірмесін жылым жанып
әкесі алыс емес қажын алар
сандақтас болар ма екен байлар жауын
бар болған соң қайта кеп деп қапырған
бұл күнде жақындады жан жамалып
әкесінің бір шығып жерді жанған
екі жетім байырға көрген жанның
көнбесем мынау іздеп ауыр тамақ
жасында жан жамылып әкесі  деп
жүргені байлор қосқа заман тұрып
үмітпен параз бермес жаман кетсе
келмедім не боп кетті қыздар едім
бір кез
